Before you turn this problem in, make sure everything runs as expected. First, **restart the kernel** (in the menubar, select Kernel $\rightarrow$ Restart) and then **run all cells** (in the menubar, select Cell $\rightarrow$ Run All).

Make sure you fill in any place that says `YOUR CODE HERE` or "YOUR ANSWER HERE", as well as your name and collaborators below:

In [2]:
NAME = "ANTONIS PRODROMOU"
ID = "238"

---

### Part A: Regular expressions (3 points)

In the exercised of this part you should NOT assume that gutenberg files are locally available in the file system.

In [3]:
import re
import nltk
nltk.download('gutenberg')
nltk.download('punkt_tab')
from nltk.corpus import gutenberg

[nltk_data] Downloading package gutenberg to /Users/anton/nltk_data...
[nltk_data]   Package gutenberg is already up-to-date!
[nltk_data] Downloading package punkt_tab to /Users/anton/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


In [4]:
gutenberg.fileids()

['austen-emma.txt',
 'austen-persuasion.txt',
 'austen-sense.txt',
 'bible-kjv.txt',
 'blake-poems.txt',
 'bryant-stories.txt',
 'burgess-busterbrown.txt',
 'carroll-alice.txt',
 'chesterton-ball.txt',
 'chesterton-brown.txt',
 'chesterton-thursday.txt',
 'edgeworth-parents.txt',
 'melville-moby_dick.txt',
 'milton-paradise.txt',
 'shakespeare-caesar.txt',
 'shakespeare-hamlet.txt',
 'shakespeare-macbeth.txt',
 'whitman-leaves.txt']

Complete the following function using the search function of the re library to find and return the year that a file in the gutenberg collection was written. Return the year as string or the empty string if not found.

In [5]:
raw = gutenberg.raw('milton-paradise.txt')
print(raw[0:300])

[Paradise Lost by John Milton 1667] 
 
 
Book I 
 
 
Of Man's first disobedience, and the fruit 
Of that forbidden tree whose mortal taste 
Brought death into the World, and all our woe, 
With loss of Eden, till one greater Man 
Restore us, and regain the blissful seat, 
Sing, Heavenly Muse, that, o


In [6]:
def get_year(file):
    raw = gutenberg.raw(file)
    # get the the first match only
    title = re.search('\[.*by.*[0-9]{4}.*]', raw)
    if title:
        date = re.search('[0-9]{4}', title.group())
        return date.group()
    else:
        return ""

In [7]:
"""Testing that the function returns the correct results"""
assert get_year('carroll-alice.txt') == '1865'
assert get_year('shakespeare-hamlet.txt') == '1599'
assert get_year('milton-paradise.txt') == '1667'


Complete the following function using the findall function of the re library to compute and report the number of times some text appears in a file of the gutenberg collection.

In [8]:
def find_mentions(text, file):
    raw = gutenberg.raw(file)
    # return all non-overlapping matches of pattern in string as a list of strings or tuples
    # re.escape makes sure that special characters in the text are escaped
    matches = re.findall(re.escape(text), raw)
    return len(matches)

find_mentions('CHAPTER ', 'carroll-alice.txt') == 12

True

In [9]:
"""Testing that the function returns the correct results"""
assert find_mentions('CHAPTER ', 'carroll-alice.txt') == 12
assert find_mentions('UNK', 'shakespeare-caesar.txt') == 0
assert find_mentions('Caesar', 'shakespeare-caesar.txt') == 227

Complete the following function to return the mean number of words in the sentences of a gutenberg file that is given as input. The number should be rounded to 2 decimals.

In [10]:
# get the sentences
sents = gutenberg.sents('austen-emma.txt')
sents_length = len(sents)
print(sents_length)

words = gutenberg.words('austen-emma.txt')
words_length = len(words)
print(words_length)

words_mean = round(words_length / sents_length, 2)
print(words_mean)

7752
192427
24.82


In [11]:
def average_sentence_length(file):
    sents = gutenberg.sents(file)
    sents_length = len(sents)
    words = gutenberg.words(file)
    words_length = len(words)
    words_mean = round(words_length / sents_length, 2)
    return words_mean

average_sentence_length('shakespeare-caesar.txt')

11.94

In [12]:
"""Testing that the function returns the correct results"""
assert average_sentence_length('shakespeare-caesar.txt') == 11.94
assert average_sentence_length('austen-emma.txt') == 24.82

## Part B: Byte-Pair Encoding (7 points)

Implement a function that returns the vocabulary learned by the byte-pair encoding algorithm given a corpus, after a number of iterations. The characters in the corpus, sorted in alphabetical order, will be the first members of the vocabulary, to be extended with as many tokens as the number of iterations given.

In [ ]:
corpus = "set new new renew reset renew"


def bpe_train(corpus, iterations):
    # Create the corpus dictionary
    # First, we'll break up the corpus into words, with leading whitespace, together with
    # their counts; no merges will be allowed to go beyond these word boundaries
    # add any unique whitespace characters
    words = re.findall(r"\b\s*\S+\b", corpus)
    print(f"Corpus C is {words}")

    # get word frequencies
    word_count_dict = {}
    for word in words:
        try:
            word_count_dict[word] += 1
        except:
            word_count_dict[word] = 1
    print(f"Word frequences is {word_count_dict}")

    # This dict will store the symbols each word contains
    word_tokens = {}
    # unpack words and their frequency
    for word, freq in word_count_dict.items():
        # From the expected results, I need to replace whitespaces with an underscore for each word
        word_with_underscore = word.replace(" ", "_")
        # Split each word into individual characters and store in the dict
        word_tokens[word] = list(word_with_underscore)

    print(word_tokens)

    vocabulary = set()
    # Initial vocabulary
    for token in word_tokens.values():
        vocabulary.update(token)
    # sort it in alphabetical order as the exercise requires
    vocabulary = sorted(list(vocabulary))
    print(f"Initial vocabulary is: {vocabulary}")

    for i in range(iterations):
        # this dict will store all the pairs of the iteration
        pairs = {}
        # unpack similar to before
        for word, freq in word_count_dict.items():
            # get a list of symbols for that word
            symbols = word_tokens[word]
            # -1 to avoid error for the last symbol
            for j in range(len(symbols) - 1):
                # store them in a tuple
                pair = (symbols[j], symbols[j + 1])
                # get(): give me the current count for this pair if it exists, otherwise return 0
                pairs[pair] = pairs.get(pair, 0) + freq

        if not pairs:
            break

        # show the pair frequencies
        print("Pair frequencies:")
        print(pairs.items())

        # most_frequent_pair = max(pairs, key=pairs.get)
        most_frequent_pair_info = sorted(pairs.items(), key=lambda x:x[1], reverse=True)[0]
        most_frequent_pair = most_frequent_pair_info[0]
        print(f"Most frequent pair to merge: {most_frequent_pair}")

        # merge the pair in every word
        new_word_tokens = {}
        for word, symbols in word_tokens.items():
            new_symbols = []
            j = 0
            while j < len(symbols):
                # if this position starts the most frequent pair, merge them
                if j < len(symbols) - 1 and (symbols[j], symbols[j + 1]) == most_frequent_pair:
                    new_symbols.append(symbols[j] + symbols[j + 1])
                    # skip the next symbol because it's merged
                    j += 2
                else:
                    new_symbols.append(symbols[j])
                    j += 1
            new_word_tokens[word] = new_symbols

        # update for next iteration
        word_tokens = new_word_tokens
        vocabulary.append(most_frequent_pair[0] + most_frequent_pair[1])

        # show how each word looks after the merge
        print("Updated tokens per word:")
        for word, tokens in word_tokens.items():
            print(word_tokens.items())

    print("\nfinal vocabulary after all merges:")
    print(list(vocabulary))
    return vocabulary

Corpus C is ['set', ' new', ' new', ' renew', ' reset', ' renew']
Word frequences is {'set': 1, ' new': 2, ' renew': 2, ' reset': 1}
{'set': ['s', 'e', 't'], ' new': ['_', 'n', 'e', 'w'], ' renew': ['_', 'r', 'e', 'n', 'e', 'w'], ' reset': ['_', 'r', 'e', 's', 'e', 't']}
Initial vocabulary is: ['_', 'e', 'n', 'r', 's', 't', 'w']
Pair frequencies:
dict_items([(('s', 'e'), 2), (('e', 't'), 2), (('_', 'n'), 2), (('n', 'e'), 4), (('e', 'w'), 4), (('_', 'r'), 3), (('r', 'e'), 3), (('e', 'n'), 2), (('e', 's'), 1)])
Most frequent pair to merge: ('n', 'e')
Updated tokens per word:
dict_items([('set', ['s', 'e', 't']), (' new', ['_', 'ne', 'w']), (' renew', ['_', 'r', 'e', 'ne', 'w']), (' reset', ['_', 'r', 'e', 's', 'e', 't'])])
dict_items([('set', ['s', 'e', 't']), (' new', ['_', 'ne', 'w']), (' renew', ['_', 'r', 'e', 'ne', 'w']), (' reset', ['_', 'r', 'e', 's', 'e', 't'])])
dict_items([('set', ['s', 'e', 't']), (' new', ['_', 'ne', 'w']), (' renew', ['_', 'r', 'e', 'ne', 'w']), (' reset', [

In [26]:
"""Testing that the function returns the correct results"""
corpus = "set new new renew reset renew"
assert set(bpe_train(corpus, 4)) == set(['_', 'e', 'n', 'r', 's', 't', 'w', 'ne', 'new', '_r', '_re'])
assert set(bpe_train(corpus, 8)) == set(['_', 'e', 'n', 'r', 's', 't', 'w', 'ne', 'new', '_r', '_re', 'se', 'set', '_new', '_renew'])
assert set(bpe_train(corpus, 9)) == set(['_', 'e', 'n', 'r', 's', 't', 'w', 'ne', 'new', '_r', '_re', 'se', 'set', '_new', '_renew', '_reset']);_
corpus = "hello world my large language model is coming after you beware"
assert set(bpe_train(corpus, 6)) == set(['_', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'l', 'm', 'n', 'o', 'r', 's', 't', 'u', 'w', 'y', 'el', '_m', '_l', '_la', 'ge', 'ng'])

Corpus C is ['set', ' new', ' new', ' renew', ' reset', ' renew']
Word frequences is {'set': 1, ' new': 2, ' renew': 2, ' reset': 1}
{'set': ['s', 'e', 't'], ' new': ['_', 'n', 'e', 'w'], ' renew': ['_', 'r', 'e', 'n', 'e', 'w'], ' reset': ['_', 'r', 'e', 's', 'e', 't']}
Initial vocabulary is: ['_', 'e', 'n', 'r', 's', 't', 'w']
Pair frequencies:
dict_items([(('s', 'e'), 2), (('e', 't'), 2), (('_', 'n'), 2), (('n', 'e'), 4), (('e', 'w'), 4), (('_', 'r'), 3), (('r', 'e'), 3), (('e', 'n'), 2), (('e', 's'), 1)])
Most frequent pair to merge: ('n', 'e')
Updated tokens per word:
dict_items([('set', ['s', 'e', 't']), (' new', ['_', 'ne', 'w']), (' renew', ['_', 'r', 'e', 'ne', 'w']), (' reset', ['_', 'r', 'e', 's', 'e', 't'])])
dict_items([('set', ['s', 'e', 't']), (' new', ['_', 'ne', 'w']), (' renew', ['_', 'r', 'e', 'ne', 'w']), (' reset', ['_', 'r', 'e', 's', 'e', 't'])])
dict_items([('set', ['s', 'e', 't']), (' new', ['_', 'ne', 'w']), (' renew', ['_', 'r', 'e', 'ne', 'w']), (' reset', [